In [ ]:
import pandas as pd
import hashlib

Load the training set

In [ ]:
Df_training_set = pd.read_csv('/Users/khaals/Desktop/word2vec/radio/training/trial1/code_train.csv', sep='$')
Df_training_set.dropna()
len(Df_training_set)

Now, I start by removing imitation noise for both the training and testing set

In [ ]:
class CLineInfo:
    def __init__(self):
        iHash = 0
        iCounter= 0
        strFileName = ""
        strCodeLine = ""
        strClassName=""
        strClassValue= ""
        strTestName = ""
        strPth=""
        iVerdict = 0

In [ ]:
lineDictionary = {}

def addHashToLine():    
    for i, strInputLine in Df_training_set.iterrows(): 
        strCodeLine = str(strInputLine['contents'])
        iHash = int(hashlib.sha1(strCodeLine.encode('utf-8')).hexdigest(), 16) % (10 ** 8) 
        Df_training_set.set_value(i,'contents',strCodeLine)
        Df_training_set.set_value(i,'hash',iHash)
        
def createDictionary():
    iLineIndex = 0
    for index, strInputLine in Df_training_set.iterrows():  
            if iLineIndex != 0:            
                lineObject = CLineInfo()
                lineObject.iHash = int(strInputLine['hash'])
                lineObject.strFileName = strInputLine['id']
                lineObject.strCodeLine = strInputLine['contents']
                lineObject.iVerdict = strInputLine['verdict']
                lineObject.strClassName = strInputLine['class_name']
                lineObject.strClassValue = strInputLine['class_value']
                lineObject.strPath = strInputLine['path']
                lineObject.strTestName = strInputLine['testName']
                lineObject.iCounter = strInputLine['line']
                # check if the line already exists in the dictionary
                if lineObject.iHash in lineDictionary:
                    old_line_verdict = lineDictionary[lineObject.iHash].iVerdict
                    # check if the new occurrence has a different verdict and that lineObject has a verdict of 1, 
                    # if so, then we relabel the dictionary line by replacing it with the lineObject
                    if (old_line_verdict != lineObject.iVerdict) and (lineObject.iVerdict == 1.0):
                          lineDictionary[lineObject.iHash] = lineObject
                    else:
                        continue #do nothing and keep the lineObject as is in the dictionary
                else:                
                    lineDictionary[lineObject.iHash] = lineObject
            iLineIndex += 1
addHashToLine()
createDictionary()
len(lineDictionary)

In [ ]:
outputFile = "./code_test_no_noise.csv"
fOutputFile = io.open(outputFile, 'w', encoding='utf-8')

In [ ]:
with open('./code_test_no_noise.csv', mode='w') as csv_file:
    header_names = ['id', 'line', 'contents', 'class_name', 'class_value', 'path' , 'testName', 'verdict'] #, 'verdict', 'hash'
    writer = csv.DictWriter(csv_file, fieldnames=header_names, quotechar='"', 
                            quoting=csv.QUOTE_ALL, delimiter='$')
    writer.writeheader()
    for hashKey, oneLineObject in lineDictionary.items():
        writer.writerow({'id': oneLineObject.strFileName, 
                         'line': str(oneLineObject.iCounter), 
                         'contents': oneLineObject.strCodeLine, 
                         'class_name': oneLineObject.strClassName, 
                         'class_value': oneLineObject.strClassValue,
                         'testName' : oneLineObject.strTestName,
                         'path': oneLineObject.strPath, 
                         'verdict': str(oneLineObject.iVerdict) }) #, 'verdict': str(oneLineObject.iVerdict),'hash': oneLineObject.iHash
                          